# BiLSTM

In [26]:
from reader import Reader
from wordembedder import Wordembedder
import nltk
import pandas as pd
import numpy as np
from sentence_splitter import SentenceSplitter
from keras import regularizers
from keras.utils import np_utils
from keras.models import Sequential
from keras.layers import LSTM, Dense, Bidirectional, Activation, Embedding
from keras.layers.embeddings import Embedding
from keras.preprocessing.text import Tokenizer
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.model_selection import KFold

In [2]:
datas = pd.read_csv('data-extraction.csv')
datas.head()

,id,texts,tags
0,0,Nuclear theory devoted major efforts since 4 d...,"['B-PHRASE', 'I-PHRASE', 'OTHER', 'B-PHRASE', ..."
1,1,The next important step might be the derivatio...,"['B-PHRASE', 'OTHER', 'B-PHRASE', 'OTHER', 'B-..."
2,2,A fluctuating vacuum is a general feature of q...,"['B-PHRASE', 'I-PHRASE', 'OTHER', 'B-PHRASE', ..."
3,3,We have presented spectrally resolved femtosec...,"['B-PHRASE', 'OTHER', 'B-PHRASE', 'I-PHRASE', ..."
4,4,"Water is the most important liquid, and the na...","['B-PHRASE', 'OTHER', 'B-PHRASE', 'OTHER', 'B-..."


In [3]:
reader = Reader()
reader.read(r"C:\Users\user\Documents\IF5282-NLP\relation-extraction\data\scienceie2017_train\train2")

In [4]:
reader.publications[0].text

'Nuclear theory devoted major efforts since 4 decades to describe thermalization in nuclear reactions, predominantly using semi-classical methods  [13,14,10], in line with similar problems in quantum liquids  [15,16]. There were attempts to develop improved molecular dynamics methods combining quantum features with a semi classical treatment of dynamical correlations  [17,18]. Still, no clear-cut quantum approach is readily available yet, in spite of numerous formal attempts [19,20,10]. The field of clusters and nano structures is far younger but fast developing in relation to the ongoing developments of lasers and imaging techniques. Semiclassical approaches were also considered in the field to include some dynamical corrections  [21,22] and could qualitatively describe dynamical processes. But such approaches are bound to simple metals with sufficiently delocalized wave functions, and thus smooth potentials justifying semiclassical approximations. The case of organic systems, in part

In [31]:
X = datas['texts']
y = datas['tags']
docs = []
for pub in X:
    docs.append(pub)
wv_doc = Wordembedder(docs, filename='embedding_matrix.npy')
encoded_docs = wv_doc.encoded_docs
vocab_size=wv_doc.vocab_size

In [45]:
encoder = LabelEncoder()
encoder.fit(y)
encoded_Y = encoder.transform(y)
y_vec = np_utils.to_categorical(encoded_Y)

X_train, X_test, y_train, y_test = train_test_split(encoded_docs, y_vec, test_size=0.2, random_state=1)
print(len(X_train), len(y_train))

280 280


In [66]:
def fit_and_eval_bilstm(X_train, y_train, X_test, y_test, X_val, y_val, vocab_size, embedding_matrix, embedding_dim, max_length):
    model = Sequential()
    model.add(Embedding(vocab_size, embedding_dim, weights=[embedding_matrix], input_length=max_length))
    model.add(Bidirectional(LSTM(units=256, recurrent_dropout=0.3, return_sequences=True, input_shape=(1, max_length, 1))))
    model.add(Bidirectional(LSTM(units=128, recurrent_dropout=0.2, dropout=0.2)))
#     model.add(Bidirectional(LSTM(units=64)))
#     model.add(Activation('softmax'))
    model.add(Dense(22, activation='softmax'))
    model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
    result = model.fit(X_train, y_train, batch_size=128, epochs=10, validation_data=(X_val, y_val), verbose=1)
    metrics = model.evaluate(x_test, y_test)
    return result, metrics[0], metrics[1]

In [67]:
kfold = KFold(10, True, random_state=1)
model_history = []
metrics = []
i=1
for train, val in kfold.split(X_train, y_train):
    print("Training on Fold: ",i)
    train_x = X_train[train]
    val_x = X_train[val]
    train_y = y_train[train]
    val_y = y_train[val]
    
    history, loss, accuracy = fit_and_eval_bilstm(train_x, train_y, X_test, y_test, val_x, val_y, 8291, wv_doc.embedding_matrix, 300, 271)
    model_history.append(history)
    metrics.append([loss, accuracy])
    i+=1

Training on Fold:  1


RuntimeError: You must compile a model before training/testing. Use `model.compile(optimizer, loss)`.